In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!gdown 1wHe0FyU3JQKD9yrLdRcpqW3_ztfFQzY2

Downloading...
From (original): https://drive.google.com/uc?id=1wHe0FyU3JQKD9yrLdRcpqW3_ztfFQzY2
From (redirected): https://drive.google.com/uc?id=1wHe0FyU3JQKD9yrLdRcpqW3_ztfFQzY2&confirm=t&uuid=c5e9f34d-976f-40ba-82cd-296692260b7b
To: /content/AWF_data.zip
100% 91.9M/91.9M [00:03<00:00, 27.0MB/s]


In [7]:
import shutil
import os
directory = "/content/dataset"
if any(os.listdir(directory)):
  print('dataset exists')
  !unzip /content/AWF_data.zip -d /content/dataset
else:
  !unzip /content/AWF_data.zip -d /content/dataset

Archive:  /content/AWF_data.zip
  inflating: /content/dataset/dataset/X_test.npy  
  inflating: /content/dataset/dataset/X_train.npy  
  inflating: /content/dataset/dataset/X_valid.npy  
  inflating: /content/dataset/dataset/y_test.npy  
  inflating: /content/dataset/dataset/y_train.npy  
  inflating: /content/dataset/dataset/y_valid.npy  


In [8]:

!gdown 15D1vov_iXPWMpGO6-xnb-M6aDXWPxtHD


Downloading...
From (original): https://drive.google.com/uc?id=15D1vov_iXPWMpGO6-xnb-M6aDXWPxtHD
From (redirected): https://drive.google.com/uc?id=15D1vov_iXPWMpGO6-xnb-M6aDXWPxtHD&confirm=t&uuid=a7e38d2b-f89e-4087-8fae-2be9225e1c19
To: /content/X_test_Unmon_NoDef.pkl
100% 801M/801M [00:06<00:00, 117MB/s]


In [9]:
mv /content/X_test_Unmon_NoDef.pkl /content/dataset/openset.pkl

In [10]:
import os
import tensorflow as tf
from tensorflow.keras import backend as K
from utility import LoadDataNoDefCW
from Model_NoDef import DFNet
import random
import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd

from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from sklearn.utils import shuffle
from tensorflow.keras.models import Model
from tensorflow.python.ops.numpy_ops import np_config
from sklearn.metrics import accuracy_score
np_config.enable_numpy_behavior()


In [11]:
def representative_data_gen_Open():
  for input_value in tf.data.Dataset.from_tensor_slices(X__open).batch(1).take(1000):
    # Model has only one input so each data point has one element.
    yield [input_value]

def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

def convert_bytes(size, unit=None):
    if unit == "KB":
        return str(round(size / 1024, 3))
    elif unit == "MB":
        return str(round(size / (1024 * 1024), 3))
    else:
        return str(size)


def Micro_F1(matrix):
    epsilon = 1e-8
    TP = 0
    FP = 0
    TN = 0

    for k in range(NB_CLASSES):
        TP += matrix[k][k]
        FP += (np.sum(Matrix, axis=0)[k] - matrix[k][k])
        TN += (np.sum(Matrix, axis=1)[k] - matrix[k][k])

    Micro_Prec = TP / (TP + FP)
    Micro_Rec = TP / (TP + TN)
    print("Micro Precision: ", Micro_Prec)
    print("Micro Recall: ", Micro_Rec)
    Micro_F1 = 2 * Micro_Prec * Micro_Rec / (Micro_Rec + Micro_Prec + epsilon)

    return Micro_F1


def New_F1_Score(Matrix):
    epsilon = 1e-8
    Column_sum = np.sum(Matrix, axis=0)
    Raw_sum = np.sum(Matrix, axis=1)

    Precision_Differences = []
    Recall_Differences = []
    for i in range(NB_CLASSES):
        Precision_Differences.append(np.abs(2 * Matrix[i][i] - Column_sum[i]))
        Recall_Differences.append(np.abs(2 * Matrix[i][i] - Raw_sum[i]))

    Precision_Differences = np.array(Precision_Differences)
    Precision_Differences_Per = Precision_Differences / (np.sum(Precision_Differences)+epsilon)
    Recall_Differences = np.array(Recall_Differences)
    Recall_Differences_Per = Recall_Differences / (np.sum(Recall_Differences)+epsilon)

    # print('Precision_Differences_Per',Precision_Differences_Per)
    # print('Recall_Differences_Per',Recall_Differences_Per)

    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    for k in range(len(Precisions)):
        Precisions[k] = (Matrix[k][k] / np.sum(Matrix, axis=0)[k])
    Precision = np.sum(np.array(Precisions) * Precision_Differences_Per)

    for k in range(len(Recalls)):
        Recalls[k] = Matrix[k][k] / (np.sum(Matrix, axis=1)[k]+ epsilon)  # *Recall_Differences_Per[k]
    Recall = np.sum(np.array(Recalls) * Recall_Differences_Per)

    print('Precision: ', Precision)
    print('Recall: ', Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score


def Macro_F1(Matrix):
    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = Matrix[k][k] / (np.sum(Matrix, axis=0)[k]+epsilon)
    # print(Precisions)

    Precision = np.average(Precisions)
    print("Precision:", Precision)

    for k in range(len(Recalls)):
        Recalls[k] = Matrix[k][k] / (np.sum(Matrix, axis=1)[k]+epsilon)

    Recall = np.average(Recalls)
    print("Recall:", Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score

In [12]:
import gc
gc.collect()

0

In [13]:
NB_EPOCH = 10   # Number of training epoch
print ("Number of Epoch: ", NB_EPOCH)
BATCH_SIZE = 128 # Batch size
VERBOSE = 2 # Output display mode
LENGTH = 1500 # Packet sequence length
OPTIMIZER = Adamax(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08) # Optimizer

NB_CLASSES = 200 # number of outputs = number of classes
INPUT_SHAPE = (LENGTH,1)

random.seed(0)

print ("Loading and preparing data for training, and evaluating the model")
X_train, y_train, X_valid, y_valid, X_test, y_test = LoadDataNoDefCW()

# Convert data as float32 type
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_valid = y_valid.astype('float32')
y_test = y_test.astype('float32')

# we need a [Length x 1] x n shape as input to the DF CNN (Tensorflow)
X_train = X_train[:, :,np.newaxis]
X_valid = X_valid[:, :,np.newaxis]
X_test = X_test[:, :,np.newaxis]

# Convert class vectors to categorical classes matrices
y_train = to_categorical(y_train, NB_CLASSES)
y_valid = to_categorical(y_valid, NB_CLASSES)
y_test = to_categorical(y_test, NB_CLASSES)

Number of Epoch:  10
Loading and preparing data for training, and evaluating the model
Loading non-defended dataset for closed-world scenario
Data dimensions:
X: Training data's shape :  (100000, 1500)
y: Training data's shape :  (100000,)
X: Validation data's shape :  (40000, 1500)
y: Validation data's shape :  (40000,)
X: Testing data's shape :  (40000, 1500)
y: Testing data's shape :  (40000,)


In [15]:
model = DFNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])
print ("Model compiled")

filepath = '/content/trained_models/AWF.hdf5'

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Start training
history = model.fit(X_train, y_train,
		batch_size=BATCH_SIZE, epochs=NB_EPOCH,
		verbose=VERBOSE, validation_data=(X_valid, y_valid), callbacks=callbacks_list)

#Start evaluating model with testing data

model_penultimate = Model(model.input, model.layers[-2].output)
model_penultimate.save('/content/trained_models/AWF_without_softmax.hdf5')
model_penultimate.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
    metrics=["accuracy"])

score_test = model_penultimate.evaluate(X_test, y_test, verbose=VERBOSE)
print("Testing closed accuracy_without_norm:", score_test[1])


del model


Model compiled
Epoch 1/10

Epoch 1: val_accuracy improved from -inf to 0.58455, saving model to /content/trained_models/AWF.hdf5
782/782 - 60s - loss: 2.9170 - accuracy: 0.3255 - val_loss: 1.5673 - val_accuracy: 0.5846 - 60s/epoch - 77ms/step
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_accuracy improved from 0.58455 to 0.85155, saving model to /content/trained_models/AWF.hdf5
782/782 - 39s - loss: 1.3216 - accuracy: 0.6564 - val_loss: 0.5846 - val_accuracy: 0.8515 - 39s/epoch - 49ms/step
Epoch 3/10

Epoch 3: val_accuracy improved from 0.85155 to 0.89935, saving model to /content/trained_models/AWF.hdf5
782/782 - 40s - loss: 0.8469 - accuracy: 0.7777 - val_loss: 0.3956 - val_accuracy: 0.8993 - 40s/epoch - 51ms/step
Epoch 4/10

Epoch 4: val_accuracy improved from 0.89935 to 0.91835, saving model to /content/trained_models/AWF.hdf5
782/782 - 40s - loss: 0.6173 - accuracy: 0.8390 - val_loss: 0.3174 - val_accuracy: 0.9183 - 40s/epoch - 51ms/step
Epoch 5/10

Epoch 5: val_accuracy improved from 0.91835 to 0.93698, saving model to /content/trained_models/AWF.hdf5
782/782 - 40s - loss: 0.4862 - accuracy: 0.8731 - val_loss: 0.2546 - val_accuracy: 0.9370 - 40s/epoch - 51ms/step
Epoch 6/10

Epoch 6: val_accuracy improved from 0.93698 to 0.95003, saving model to /co

1250/1250 - 7s - loss: 16.0270 - accuracy: 0.9701 - 7s/epoch - 6ms/step
Testing closed accuracy_without_norm: 0.9701499938964844


In [16]:
import _pickle as cPickle
dataset_dir = "/content/dataset/"
with open( dataset_dir+"openset.pkl","rb") as f:
        X_open = cPickle.load(f,encoding='latin1')
X_open = np.array(X_open)
X_open=X_open[:,:1500]
print(X_open.shape)
X_open = X_open.astype('float32')
X__open = X_open[:,:,np.newaxis]
y_open = np.array([NB_CLASSES]*len(X_open))
y_open = y_open.astype('uint8')

(20000, 1500)


In [17]:
def representative_data_gen():
  for input_value in tensorflow.data.Dataset.from_tensor_slices(X_train_Rep).batch(1).take(1000):
    # Model has only one input so each data point has one element.
    yield [input_value]

Block=1000
X_train_Rep,y_train_Rep=shuffle(X_train, y_train)
y_train=np.argmax(y_train, axis=1)
y_valid=np.argmax(y_valid, axis=1)
y_test=np.argmax(y_test, axis=1)

X_test=np.squeeze(X_test, axis=-1)

TF_LITE_MODEL_FILE_NAME = "tf_lite_model_fullint.tflite"

converter = tf.lite.TFLiteConverter.from_keras_model(model_penultimate)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_types = [tf.float16]       # COnvert to 16 bit
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.uint8

tflite_model = converter.convert()

tflite_model_name = "/content/trained_models/"+TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

# print("Original Model Size:",convert_bytes(get_file_size('/content/trained_models/IoT_without_softmax.hdf5'), "KB"),"KB")
# print("Quantized model size:",convert_bytes(get_file_size(tflite_model_name), "KB"),"KB")
# print()

#X_test=np.squeeze(X_test, axis=-1)
interpreter = tf.lite.Interpreter(model_path = tflite_model_name)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.resize_tensor_input(input_details[0]['index'], (Block, LENGTH, 1))       #change the input shape according to the test dataset dimenssions
interpreter.resize_tensor_input(output_details[0]['index'], (Block, NB_CLASSES))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_scale,input_zero_point  = input_details[0]["quantization"]


for i in range(len(X_test)):
  X_test[i] = X_test[i] / input_scale + input_zero_point

X_test=X_test[:,:,np.newaxis]
X_test = X_test.astype('int8')
tflite_model_predictions_test=[]

for i in range(5):
  interpreter.set_tensor(input_details[0]['index'], X_test[i*Block:(i+1)*Block])
  interpreter.invoke()
  Mod_Prediction=interpreter.get_tensor(output_details[0]['index'])
  if i==0:
    tflite_model_predictions_test=Mod_Prediction
  else:
    tflite_model_predictions_test=np.concatenate((tflite_model_predictions_test,Mod_Prediction),axis=0)
  print(i,"Done")

acc_ = accuracy_score(np.argmax(tflite_model_predictions_test,axis=1), y_test[:len(tflite_model_predictions_test)])
print('Test accuracy TFLITE model :', acc_)

del X_test
gc.collect()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


0 Done
1 Done
2 Done
3 Done
4 Done
Test accuracy TFLITE model : 0.9776


3770

In [18]:

for i in range(NB_CLASSES):
  variable_name = f"Mean_{i}"
  locals()[variable_name]=np.array([0] * NB_CLASSES)

print(X_train.shape)
X_train=np.squeeze(X_train, axis=-1)
X_valid=np.squeeze(X_valid, axis=-1)
print(X_train.shape)

for i in range(len(X_train)):
  X_train[i] = X_train[i] / input_scale + input_zero_point

X_train = X_train[:, :,np.newaxis]
X_train = X_train.astype('int8')

# interpreter = tf.lite.Interpreter(model_path = tflite_model_name)
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()
# interpreter.resize_tensor_input(input_details[0]['index'], (Block, LENGTH, 1))
# interpreter.resize_tensor_input(output_details[0]['index'], (Block, NB_CLASSES))
# interpreter.allocate_tensors()


tflite_model_predictions = []
for i in range(20):
  interpreter.set_tensor(input_details[0]['index'], X_train[i*Block:(i+1)*Block])
  interpreter.invoke()
  Mod_Prediction=interpreter.get_tensor(output_details[0]['index'])
  if i==0:
    tflite_model_predictions=Mod_Prediction
  else:
    tflite_model_predictions=np.concatenate((tflite_model_predictions,Mod_Prediction),axis=0)
  print(i,"Done")

del X_train

count=[0]*NB_CLASSES
txt_O = "Mean_{Class1:.0f}"
Means={}
for i in range(NB_CLASSES):
  Means[txt_O.format(Class1=i)]=np.array([0]*NB_CLASSES)

for i in range(len(tflite_model_predictions)):
  k=np.argmax(tflite_model_predictions[i])
  if (np.argmax(tflite_model_predictions[i])==y_train[i]):
    Means[txt_O.format(Class1=y_train[i])]=Means[txt_O.format(Class1=y_train[i])]+tflite_model_predictions[i]
    count[y_train[i]]+=1
print("Counts: ",count)

Mean_Vectors=[]
for i in range(NB_CLASSES):
  Means[txt_O.format(Class1=i)]=Means[txt_O.format(Class1=i)]/count[i]
  Mean_Vectors.append(Means[txt_O.format(Class1=i)])

Mean_vectors=np.array(Mean_Vectors)


Mean_vectors = np.array(Mean_Vectors)
np.save('/content/temp_variables/Mean_vectors.npy', Mean_vectors, allow_pickle=True)

# interpreter = tf.lite.Interpreter(model_path = tflite_model_name)

# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

for i in range(len(X_valid)):
  X_valid[i] = X_valid[i] / input_scale + input_zero_point

# interpreter = tf.lite.Interpreter(model_path = tflite_model_name)
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()
# interpreter.resize_tensor_input(input_details[0]['index'], (Block, LENGTH, 1))
# interpreter.resize_tensor_input(output_details[0]['index'], (Block, NB_CLASSES))
# interpreter.allocate_tensors()

X_valid = X_valid[:, :,np.newaxis]
X_valid = X_valid.astype('int8')

tflite_model_predictions = []
for i in range(8):
  interpreter.set_tensor(input_details[0]['index'], X_valid[i*Block:(i+1)*Block])
  interpreter.invoke()
  Mod_Prediction=interpreter.get_tensor(output_details[0]['index'])
  if i==0:
    tflite_model_predictions=Mod_Prediction
  else:
    tflite_model_predictions=np.concatenate((tflite_model_predictions,Mod_Prediction),axis=0)
  print(i,"Done")

del X_valid
gc.collect()

(100000, 1500, 1)
(100000, 1500)
0 Done
1 Done
2 Done
3 Done
4 Done
5 Done
6 Done
7 Done
8 Done
9 Done
10 Done
11 Done
12 Done
13 Done
14 Done
15 Done
16 Done
17 Done
18 Done
19 Done
Counts:  [83, 84, 108, 91, 113, 80, 106, 100, 96, 112, 104, 98, 89, 105, 94, 104, 102, 111, 102, 109, 90, 103, 102, 93, 78, 105, 93, 93, 104, 117, 92, 119, 96, 80, 105, 94, 85, 109, 100, 95, 110, 102, 85, 92, 103, 95, 101, 98, 76, 98, 108, 85, 112, 89, 90, 107, 90, 96, 96, 103, 102, 104, 104, 106, 106, 90, 109, 102, 105, 108, 116, 83, 93, 102, 102, 94, 102, 81, 89, 99, 110, 104, 105, 101, 91, 110, 92, 101, 91, 110, 100, 104, 104, 110, 97, 107, 82, 89, 104, 99, 101, 109, 112, 103, 89, 85, 94, 106, 103, 94, 101, 103, 91, 103, 108, 104, 103, 102, 80, 81, 81, 91, 83, 97, 96, 90, 96, 88, 86, 85, 92, 109, 89, 114, 95, 98, 99, 90, 109, 124, 87, 104, 102, 112, 102, 92, 114, 89, 93, 107, 99, 104, 90, 89, 89, 89, 100, 96, 85, 113, 113, 78, 108, 90, 89, 106, 92, 96, 91, 101, 104, 88, 107, 100, 134, 107, 89, 110, 92, 

0

In [19]:
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]


Indexes=[]
for i in range(NB_CLASSES):
  Indexes.append([])

Values={}
for i in range(NB_CLASSES):
  Values[i]=[0]*NB_CLASSES


for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]):
        Values[y_valid[i]][k]+=np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])-dist

for i in range(NB_CLASSES):
  Tot=0
  for l in range(30):
      Min=min(Values[i])
      Tot+=Min
      Indexes[i].append(Values[i].index(Min))
      Values[i][Values[i].index(Min)]=np.inf

Indexes=np.array(Indexes)

np.save('/content/temp_variables/Values.npy',np.array(Values))
np.save('/content/temp_variables/Indexes.npy',Indexes)

print()
##################################################################################################
print()
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Distances[txt_1.format(Class1=y_valid[i])].append(dist)


TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.8)]



Threasholds_1=np.array(TH)
np.save('/content/temp_variables/Threasholds_1.npy',Threasholds_1)
print("Thresholds for method 1 calculated")
##################################################################################################
print()
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]) and k in Indexes[y_valid[i]]:
        Tot+=(np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])-dist)
    Distances[txt_1.format(Class1=y_valid[i])].append(Tot)

TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.8)]



Threasholds_2=np.array(TH)
np.save('/content/temp_variables/Threasholds_2.npy',Threasholds_2)


print("Thresholds for method 2 calculated")

##################################################################################################
print()
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]) and k in Indexes[y_valid[i]]:
        Tot+=np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])
    Tot=dist/Tot
    Distances[txt_1.format(Class1=y_valid[i])].append(Tot)

TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.2)]

Threasholds_3=np.array(TH)
np.save('/content/temp_variables/Threasholds_3.npy',Threasholds_3)
print("Thresholds for method 3 calculated")



Thresholds for method 1 calculated

Thresholds for method 2 calculated

Thresholds for method 3 calculated


In [20]:

for i in range(len(X_open)):
  X_open[i] = X_open[i] / input_scale + input_zero_point

X_open = X_open[:, :,np.newaxis]
X_open = X_open.astype('int8')

for i in range(10):
  interpreter.set_tensor(input_details[0]['index'], X_open[i*Block:(i+1)*Block])
  interpreter.invoke()
  Mod_Prediction=interpreter.get_tensor(output_details[0]['index'])
  if i==0:
    tflite_model_predictions_open=Mod_Prediction
  else:
    tflite_model_predictions_open=np.concatenate((tflite_model_predictions,Mod_Prediction),axis=0)
  print(i,"Done")

del X_open
gc.collect()

0 Done
1 Done
2 Done
3 Done
4 Done
5 Done
6 Done
7 Done
8 Done
9 Done


0

In [21]:
print()
print("############## Distance Method 1 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions_test)):

    d=np.argmax(tflite_model_predictions_test[i], axis=0)
    if np.linalg.norm(tflite_model_predictions_test[i]-Mean_vectors[d])>Threasholds_1[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes=np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test[:len(prediction_classes)])
print('Test accuracy Normal model_Closed_set :', acc_Close)

prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):

    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    if np.linalg.norm(tflite_model_predictions_open[i]-Mean_vectors[d])>=Threasholds_1[d]:
      prediction_classes_open.append(NB_CLASSES)

    else:
      prediction_classes_open.append(d)

prediction_classes_open=np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open[:len(prediction_classes_open)])
print('Test accuracy Normal model_Open_set :', acc_Open)

y_test=y_test[:len(prediction_classes)]
y_open=y_open[:len(prediction_classes_open)]

Matrix=[]
for i in range(NB_CLASSES+1):
  Matrix.append(np.zeros(NB_CLASSES+1))


for i in range(len(y_test)):
  Matrix[y_test[i]][prediction_classes[i]]+=1

for i in range(len(y_open)):
  Matrix[y_open[i]][prediction_classes_open[i]]+=1

F1_Score=New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
print("Micro")

F1_Score=Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
print("Macro")
F1_Score=Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)




#############################################################################################################
print()
print("############## Distance Method 2 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions_test)):
    d=np.argmax(tflite_model_predictions_test[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_test[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_test[i])-dist

    if Tot<Threasholds_2[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes=np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test[:len(prediction_classes)])
print('Test accuracy Normal model_Closed_set :', acc_Close)

prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):
    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    dist = np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_open[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(d) and k in Indexes[d]:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_open[i])-dist

    if Tot<=Threasholds_2[d]:
      prediction_classes_open.append(NB_CLASSES)
    else:
      prediction_classes_open.append(d)

prediction_classes_open=np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open[:len(prediction_classes_open)])
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix=[]
for i in range(NB_CLASSES+1):
  Matrix.append(np.zeros(NB_CLASSES+1))


for i in range(len(y_test)):
  Matrix[y_test[i]][prediction_classes[i]]+=1

for i in range(len(y_open)):
  Matrix[y_open[i]][prediction_classes_open[i]]+=1

F1_Score=New_F1_Score(Matrix)

print("Average Novel F1_Score: ", F1_Score)

print()
print("Micro")
F1_Score=Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
print("Macro")
F1_Score=Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)



#############################################################################################################
print()
print("############## Distance Method 3 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions_test)):
    d=np.argmax(tflite_model_predictions_test[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_test[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_test[i])

    Tot=dist/Tot
    if Tot>Threasholds_3[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes=np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test[:len(prediction_classes)])
print('Test accuracy Normal model_Closed_set :', acc_Close)


prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):
    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_open[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(d) and k in Indexes[d]:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_open[i])
    Tot=dist/Tot
    if Tot>=Threasholds_3[d]:
      prediction_classes_open.append(NB_CLASSES)

    else:
      prediction_classes_open.append(d)

prediction_classes_open=np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open[:len(prediction_classes_open)])
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix=[]
for i in range(NB_CLASSES+1):
  Matrix.append(np.zeros(NB_CLASSES+1))


for i in range(len(y_test)):
  Matrix[y_test[i]][prediction_classes[i]]+=1

for i in range(len(y_open)):
  Matrix[y_open[i]][prediction_classes_open[i]]+=1

F1_Score=New_F1_Score(Matrix)

print("Average Novel F1_Score: ", F1_Score)

print()
print("Micro")
F1_Score=Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
print("Macro")
F1_Score=Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)


############## Distance Method 1 #################################

Test accuracy Normal model_Closed_set : 0.7874
Test accuracy Normal model_Open_set : 0.31377777777777777
Precision:  0.30752538503285365
Recall:  0.804659011787169
Average novel F1_Score:  0.4449856892482909

Micro
Micro Precision:  0.38887791386803633
Micro Recall:  0.7874
Average Micro F1_Score:  0.5206294586789234

Macro
Precision: 0.10417420432435741
Recall: 0.09842499999507874
Average Macro F1_Score:  0.10121802391539252

############## Distance Method 2 #################################

Test accuracy Normal model_Closed_set : 0.9766
Test accuracy Normal model_Open_set : 0.84
Precision:  0.748295065117274
Recall:  0.9770898027187136
Average Novel F1_Score:  0.8475227607974043

Micro
Micro Precision:  0.7611847233047545
Micro Recall:  0.9766
Average Micro F1_Score:  0.8555409499607548

Macro
Precision: 0.12022514175732357
Recall: 0.12207499999389626
Average Macro F1_Score:  0.1211430044365836

############## Dist